In [2]:
import numpy as np
from scipy.special import jacobi
from math import factorial, sqrt, sin, cos, radians

In [3]:
theta = radians(45)
l = 1
m = 1
n = 1

### calc reduced matrix elements (d_lmnop)
d_lmn = (-1)**(m-n)*sqrt(factorial(l-m)*factorial(l+m)/(factorial(l-n)*factorial(l+n)))*(sin(theta/2))**(m-n)*(cos(theta/2))**(m+n)*jacobi(l-m, m-n, m+n)(cos(theta))
print(d_lmn)

0.8535533905932737


In [2]:
l = 32
dim = 2*l + 1

term_1 = np.zeros((dim*dim)) ### term_1 == (-1)**(m-n)
term_2 = np.zeros_like(term_1) ### term_2 == sqrt(factorial(l-m)*factorial(l+m)/(factorial(l-n)*factorial(l+n)))
term_3 = np.zeros_like(term_1) ### term_3 == (m-n) for (sin(theta/2))**(m-n)
term_4 = np.zeros_like(term_1) ### term_4 == (m+n) for (cos(theta/2))**(m+n)
# term_5 = np.empty_like(term_1) ### term_5 == jacobi(l-m, m-n, m+n) for jacobi(l-m, m-n, m+n)(cos(theta))
term_5 = [np.array([None])]

count = -1

for m in np.arange(l, -l-1, -1):
    # for n in np.arange(m, l):
    for n in np.arange(l, -l-1, -1):
        count += 1
        if (m-n >= 0) and (m+n >= 0):
            term_1[count] = (-1)**(m-n)
            term_2[count] = sqrt(factorial(l-m)*factorial(l+m)/(factorial(l-n)*factorial(l+n)))
            term_3[count] = (m-n)
            term_4[count] = (m+n)
            term_5.append(jacobi(l-m, m-n, m+n))
        else:
            term_5.append(None)

term_5 = term_5[1:]

theta = radians(45)
d_lmnop = np.zeros((dim*dim))

for i in np.arange(d_lmnop.size):
    if term_5[i] is None:
        d_lmnop[i] = 0
    else:
        d_lmnop[i] = term_1[i]*term_2[i]*(sin(theta/2)**term_3[i])*(cos(theta/2)**term_4[i])*term_5[i](cos(theta))


In [2]:
###################

In [2]:
def precalc_reduced_terms(l):

    dim = 2*l + 1

    term_0 = np.zeros((dim*dim)) ### term_0 == (-1)**(m-n)
    term_1 = np.zeros_like(term_0) ### term_1 == sqrt(factorial(l-m)*factorial(l+m)/(factorial(l-n)*factorial(l+n)))
    term_2 = np.zeros_like(term_0) ### term_2 == (m-n) for (sin(theta/2))**(m-n)
    term_3 = np.zeros_like(term_0) ### term_3 == (m+n) for (cos(theta/2))**(m+n)
    term_4 = [np.array([None], dtype=object)] ### term_4 == jacobi(l-m, m-n, m+n) for jacobi(l-m, m-n, m+n)(cos(theta))
    term_m = np.zeros_like(term_0)
    term_n = np.zeros_like(term_0)

    count = -1

    for m in np.arange(l, -l-1, -1):
        # for n in np.arange(m, l):
        for n in np.arange(l, -l-1, -1):
            count += 1
            term_m[count] = m
            term_n[count] = n
            if (m-n >= 0) and (m+n >= 0):
                term_0[count] = (-1)**(m-n)
                term_1[count] = sqrt(factorial(l-m)*factorial(l+m)/(factorial(l-n)*factorial(l+n)))
                term_2[count] = (m-n)
                term_3[count] = (m+n)
                term_4.append(jacobi(l-m, m-n, m+n))
                # term_4 = np.append(term_4, jacobi(l-m, m-n, m+n))
            else:
                # term_4.append(None)
                term_4.append(0)
                # term_4 = np.append(term_4, None)

    term_4 = term_4[1:]

    return [term_0, term_1, term_2, term_3, term_4, term_m, term_n]

In [3]:
def calc_reduced_mtx_d(theta, terms):
    # d_lmnop = np.zeros_like(terms[0])

    d_lmnop = np.asarray([terms[0][i]*terms[1][i]*(sin(theta/2)**terms[2][i])*(cos(theta/2)**terms[3][i])*terms[4][i](cos(theta)) if terms[4][i] !=0 else 0 for (i,_) in enumerate(terms[0])])

    return d_lmnop

In [21]:
def calc_Wigner_active_rotation_mtx_D(phi1, PHI, phi2, terms):

    d_lmnop = calc_reduced_mtx_d(PHI, terms)

    D_LMNOP = np.asarray([np.exp(-1j*terms[5][i]*phi1)*d_lmnop[i]*np.exp(-1j*terms[6][i]*phi2) if d_lmnop[i] !=0 else 0 for (i,_) in enumerate(d_lmnop)])

    return D_LMNOP

In [12]:
l = 32
dim = 2*l + 1
# d_lmnop = np.zeros((dim*dim))
l_vector = np.arange(1,l+1)

# terms = precalc_reduced_terms(l)
vect_precalc = np.vectorize(precalc_reduced_terms)
all_terms = vect_precalc(l_vector)

terms = all_terms[0]
for i,_ in enumerate(all_terms):
    terms[0] = np.append(terms[0], all_terms[i][0])
    terms[1] = np.append(terms[1], all_terms[i][1])
    terms[2] = np.append(terms[2], all_terms[i][2])
    terms[3] = np.append(terms[3], all_terms[i][3])
    terms[4] = np.append(terms[4], all_terms[i][4])
    terms[5] = np.append(terms[5], all_terms[i][5])
    terms[6] = np.append(terms[6], all_terms[i][6])


/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


In [13]:
theta = radians(45)

d_lmnop = calc_reduced_mtx_d(theta, terms)
print(d_lmnop.shape)

(47913,)


In [14]:
theta = radians(45)

d_lmnop = np.zeros_like(terms[0])

for i,_ in enumerate(d_lmnop):
    if terms[4][i] == 0:
        d_lmnop[i] = 0
    else:
        # d_lmnop[i] = term_1[i]*term_2[i]*(sin(theta/2)**term_3[i])*(cos(theta/2)**term_4[i])*term_5[i](cos(theta))
        d_lmnop[i] = terms[0][i]*terms[1][i]*(sin(theta/2)**terms[2][i])*(cos(theta/2)**terms[3][i])*terms[4][i](cos(theta))

print(d_lmnop.shape)

(47913,)


In [20]:
phi1 = radians(30)
PHI = radians(45)
phi2 = radians(90)

l = 1
m = 1
n = 1

d_lmn = (-1)**(m-n)*sqrt(factorial(l-m)*factorial(l+m)/(factorial(l-n)*factorial(l+n)))*(sin(PHI/2))**(m-n)*(cos(PHI/2))**(m+n)*jacobi(l-m, m-n, m+n)(cos(PHI))
D_LMN = np.exp(-1j*m*phi1)*d_lmn*np.exp(-1j*n*phi2)

print(D_LMN)

(-0.42677669529663675-0.7391989197401166j)


In [22]:
phi1 = radians(30)
PHI = radians(45)
phi2 = radians(90)

D_LMNOP =  calc_Wigner_active_rotation_mtx_D(phi1, PHI, phi2, terms)


In [23]:
print(D_LMNOP.shape)

(47913,)
